In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F 
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch import optim

In [14]:
class ResBlk(nn.Module):
    """
    resnet block
    """
    def __init__(self,ch_in,ch_out,stride=1):
        super(ResBlk,self).__init__()
        
        self.conv1=nn.Conv2d(ch_in,ch_out,kernel_size=3,stride=stride,padding=1)
        self.bn1=nn.BatchNorm2d(ch_out)
        self.conv2=nn.Conv2d(ch_out,ch_out,kernel_size=3,stride=1,padding=1)
        self.bn2=nn.BatchNorm2d(ch_out)
        
        self.extra=nn.Sequential()
        if ch_out != ch_in:
            # [b,ch_in,h,w] => [b,ch_out,h,w]
            self.extra=nn.Sequential(
                nn.Conv2d(ch_in,ch_out,kernel_size=1,stride=stride),
                nn.BatchNorm2d(ch_out)
            )
            
    def forward(self,x):
        out=F.relu(self.bn1(self.conv1(x)))
        print('conv1-bn1:',out.shape)
        out=self.bn2(self.conv2(out))
        print('conv2-bn2:',out.shape)
        # short cut
        # extra module: [b,ch_in,h,w] => [b,ch_out,h,w]
        # element-wise add:
        out=self.extra(x)+out
        out=F.relu(out)
        print('extra:',out.shape)
        
        return out

In [15]:
class ResNet18(nn.Module):
    
    def __init__(self):
        super(ResNet18,self).__init__()
        
        self.conv1=nn.Sequential(
            nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm2d(64)
        )
        # follwed 4 blocks
        # [b,64,h,w] => [b,128,h,w]
        self.blk1=ResBlk(64,128,stride=2)
        # [b,128,h,w] => [b,256,h,w]
        self.blk2=ResBlk(128,256,stride=2)
        # [b,256,h,w] => [b,512,h,w]
        self.blk3=ResBlk(256,512,stride=2)
        # [b,512,h,w] => [b,512,h,w]
        self.blk4=ResBlk(512,512,stride=1)
        
        self.outlayer=nn.Linear(512*4*4,10)
        
    def forward(self,x):
        x=F.relu(self.conv1(x))
        print('conv1:',x.shape)
        print('\n')
        # [b,64,h,w] => [b,1024,h,w]
        x=self.blk1(x)
        print('blk1:',x.shape)
        print('\n')
        x=self.blk2(x)
        print('blk2:',x.shape)
        print('\n')
        x=self.blk3(x)
        print('blk3:',x.shape)
        print('\n')
        x=self.blk4(x)
        print('blk4:',x.shape)
        
#         print('after conv:',x.shape) # [b,512,4,4]
        # [b,512,h,w] => [b,512,1,1] 取像素点均值
#         x=F.adaptive_avg_pool2d(x,[1,1])
#         print('after pool:',x.shape)
        x=x.view(x.size(0),-1)
        x=self.outlayer(x)
        
        return x

In [16]:
def main():

#     blk = ResBlk(64, 128, stride=2)
#     tmp = torch.randn(2, 64, 32, 32)
#     out = blk(tmp)
#     print('block:', out.shape)

    x = torch.randn(2, 3, 32, 32)
    model = ResNet18()
    out = model(x)
    print('resnet:', out.shape)

if __name__=='__main__':
    main()

conv1: torch.Size([2, 64, 32, 32])


conv1-bn1: torch.Size([2, 128, 16, 16])
conv2-bn2: torch.Size([2, 128, 16, 16])
extra: torch.Size([2, 128, 16, 16])
blk1: torch.Size([2, 128, 16, 16])


conv1-bn1: torch.Size([2, 256, 8, 8])
conv2-bn2: torch.Size([2, 256, 8, 8])
extra: torch.Size([2, 256, 8, 8])
blk2: torch.Size([2, 256, 8, 8])


conv1-bn1: torch.Size([2, 512, 4, 4])
conv2-bn2: torch.Size([2, 512, 4, 4])
extra: torch.Size([2, 512, 4, 4])
blk3: torch.Size([2, 512, 4, 4])


conv1-bn1: torch.Size([2, 512, 4, 4])
conv2-bn2: torch.Size([2, 512, 4, 4])
extra: torch.Size([2, 512, 4, 4])
blk4: torch.Size([2, 512, 4, 4])
resnet: torch.Size([2, 10])


In [ ]:
def main():
    batchsz=32 
    epochs=1000
    
    cifar_train=datasets.CIFAR10('./data/cifar',train=True,download=True,
                                transform=transforms.Compose([
                                    transforms.Resize((32,32)),
                                    transforms.ToTensor()
                                ]))
    cifar_train=DataLoader(cifar_train,batch_size=batchsz,shuffle=True)
    
    cifar_test=datasets.CIFAR10('./data/cifar',train=False,download=True,
                                transform=transforms.Compose([
                                    transforms.Resize((32,32)),
                                    transforms.ToTensor()
                                ]))
    cifar_test=DataLoader(cifar_test,batch_size=batchsz,shuffle=True)
    
    x,label=iter(cifar_train).next()
    print('x:',x.shape,'label:',label.shape)
    
    device=torch.device('cuda')
    model=ResNet18().to(device)
    
    criteon=nn.CrossEntropyLoss().to(device)
    optimizer=optim.Adam(model.parameters(),lr=1e-3)
    print(model)
    
    for epoch in range(epochs):
        
        model.train() 
        for batchidx,(x,label) in enumerate(cifar_train):
            # [b,3,32,32]
            x,label=x.to(device),label.to(device)
            
            logits=model(x)
            # logits: [b,10]
            # label: [b]
            # loss: tensor scaler
            loss=criteon(logits,label)
            
            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print('epoch={}, loss={}'.format(epoch,loss.item()))
        
        model.eval()
        with torch.no_grad():
            # test
            total_correct=0 
            total_num=0 
            for x,label in cifar_test:
                x,label=x.to(device),label.to(device)
                
                # [b,10]
                logits=model(x)
                # [b]
                pred=logits.argmax(dim=1)
                # [b] vs [b] => scaler tensor
                correct=torch.eq(pred,label).float().sum().item()
                total_correct+=correct
                total_num+=x.size(0)
                # print(correct)
                
            acc=total_correct / total_num
            print('acc:',acc)

In [ ]:
if __name__=='__main__':
    main()